In [1]:
import os
import json
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras.models import load_model

import wandb
from wandb.keras import WandbCallback

In [2]:
genre_dict = {
        0 : "blues",
        1 : "classical",
        2 : "country",
        3 : "disco",
        4 : "hiphop",
        5 : "jazz",
        6 : "metal",
        7 : "pop",
        8 : "reggae",
        9 : "rock",
        }
        
# Path to labels and processed data file, json format.
JSON_PATH = 'data.json'

In [3]:
run = wandb.init()
artifact = run.use_artifact('msaintfelix/Genre-Classifier-Optimization/CNN_Genre_Classifier_ebw8rm2s:v0', type='model')
artifact_dir = artifact.download()
model = load_model(artifact_dir + '/my_model.h5')

wandb: Currently logged in as: msaintfelix. Use `wandb login --relogin` to force relogin


In [4]:
def load_data(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()

    # Let's load our data into numpy arrays for TensorFlow compatibility.
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

In [5]:
def prepare_datasets(inputs, targets, split_size):
    
    # Creating a validation set and a test set with fixed random_state
    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=split_size, random_state=42)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size=split_size, random_state=42)
    
    # Our CNN model expects 3D input shape.
    inputs_train = inputs_train[..., np.newaxis]
    inputs_val = inputs_val[..., np.newaxis]
    inputs_test = inputs_test[..., np.newaxis]
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [6]:
inputs, targets = load_data(json_path=JSON_PATH)     
Xtrain, Xval, Xtest, ytrain, yval, ytest = prepare_datasets(inputs, targets, 0.2)

In [7]:
pred_test = model.predict(Xtest)

In [8]:
wandb.log({"roc" : wandb.plot.roc_curve(ytest.tolist(), pred_test, labels=genre_dict, classes_to_plot=None)})

In [9]:
predictions_test = pred_test.argmax(axis=1)

In [10]:
wandb.log({"conf_mat" : wandb.plot.confusion_matrix(
                            probs=None,
                            preds=predictions_test.tolist(), 
                            y_true=ytest.tolist(),
                            class_names=genre_dict)
              })

In [11]:
run.finish()

In [12]:
accuracy_score(ytest.tolist(), predictions_test.tolist())

0.8211382113821138

In [13]:
confusion_matrix(ytest.tolist(), predictions_test.tolist())

array([[125,   0,   5,   0,   2,   1,   5,   6,   2,   3],
       [  0, 160,   0,   4,   0,   0,   4,   1,   0,   0],
       [  6,   3, 136,   1,   1,   0,   1,   3,   2,   2],
       [  0,   1,   0, 147,   1,   0,   1,   0,   4,   5],
       [  6,   0,  13,   6, 103,   1,   2,  24,   4,   1],
       [  0,   0,   0,   2,   0, 151,   1,   0,   1,   7],
       [  5,  18,   6,   8,   3,   1,  90,   7,   5,   6],
       [ 10,   8,   4,   7,   1,   0,   1, 141,   0,   1],
       [  3,   2,  11,   8,   2,   2,  13,   0, 116,  12],
       [  0,   0,   0,   4,   0,   6,   0,   0,   0, 144]])